In [2]:
import csv
import json
import base64

from urllib.request import urlopen

key = "55228d9d4f216ad5972abd1be3c4c2e3"

In [3]:
#get the first col of the csv file
def get_bill_ids(filePath):
    bill_ids = []
    with(open(filePath, 'r')) as f:
        reader = csv.reader(f)
        for row in reader:
            bill_ids.append(row[0])
    return bill_ids        

In [4]:
bill_ids = get_bill_ids("VA/2024-2024_1st_Special_Session/csv/bills.csv")

In [5]:
bill_ids[:10]

['bill_id',
 '1877734',
 '1877735',
 '1882815',
 '1883989',
 '1877732',
 '1877737',
 '1877730',
 '1877736',
 '1877727']

In [44]:
def get_text_from_bill_id(bill_id):
    urlGetBill = f"https://api.legiscan.com/?key={key}&op=getBill&id={bill_id}"
    r1 = urlopen(urlGetBill).read().decode()
    data1 = json.loads(r1)
    if data1["status"] == "ERROR":
        return None
    if data1["bill"]["texts"] == []:
        return ""
    else:
        doc_id = data1["bill"]["texts"][0]["doc_id"]
        urlGetText = f"https://api.legiscan.com/?key={key}&op=getBillText&id={doc_id}"
        r2 = urlopen(urlGetText).read().decode()
        data2 = json.loads(r2)
        textEncoded = data2["text"]["doc"]
        return textEncoded

In [45]:
bill_ids[1]

'1877734'

In [53]:
data = get_text_from_bill_id(bill_ids[5])
base64.b64decode(data).decode('utf-8')

with open("sampleText.txt", "w") as f:
    f.write(base64.b64decode(data).decode('utf-8'))


In [51]:
#Create a CSV file with the bill_id and the encoded text
bill_texts = []
bill_texts.append(["bill_id", "text"])
for bill_id in bill_ids: #skip the first line since its the header
    if bill_id == "bill_id":
        continue
    billText = get_text_from_bill_id(bill_id)
    bill_texts.append([bill_id, billText]) #If empty string then text was unavailable on LegiScan

with open("bill_texts.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(bill_texts)
        
